In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-02-06 06:37:33.221681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 06:37:33.467078: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 06:37:37.868571: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '1990-01-01','2017-12-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1
#kospi_train['Change7'] = kospi_train['Close']/kospi_train['Close'].shift(7) - 1
#kospi_train['Change15'] = kospi_train['Close']/kospi_train['Close'].shift(15) - 1
#kospi_train['Change30'] = kospi_train['Close']/kospi_train['Close'].shift(30) - 1
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1996-12-11  705.989990  709.479980  704.429993  704.679993  704.679993   
1996-12-12  705.109985  706.010010  688.739990  689.380005  689.380005   
1996-12-13  690.440002  695.719971  677.640015  689.070007  689.070007   
1996-12-16  686.969971  686.969971  667.710022  673.919983  673.919983   
1996-12-17  675.349976  680.090027  660.390015  663.349976  663.349976   

             Volume   Change1  
Date                           
1996-12-11  28000.0       NaN  
1996-12-12  25900.0 -0.021712  
1996-12-13  26500.0 -0.000450  
1996-12-16  22800.0 -0.021986  
1996-12-17  31600.0 -0.015684  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')


In [4]:
kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1996-12-12  705.109985  706.010010  688.739990  689.380005  689.380005   
1996-12-13  690.440002  695.719971  677.640015  689.070007  689.070007   
1996-12-16  686.969971  686.969971  667.710022  673.919983  673.919983   
1996-12-17  675.349976  680.090027  660.390015  663.349976  663.349976   
1996-12-18  662.070007  671.429993  655.469971  661.390015  661.390015   

             Volume   Change1  
Date                           
1996-12-12  25900.0 -0.021712  
1996-12-13  26500.0 -0.000450  
1996-12-16  22800.0 -0.021986  
1996-12-17  31600.0 -0.015684  
1996-12-18  31500.0 -0.002955  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')


In [5]:
kospi_train_close = kospi_train['Close']
kospi_train_c1 = kospi_train['Change1']
#kospi_train_c7 = kospi_train['Change7']
#kospi_train_c15 = kospi_train['Change15']
#kospi_train_c30 = kospi_train['Change30']
print(kospi_train_close, kospi_train_c1)

Date
1996-12-12     689.380005
1996-12-13     689.070007
1996-12-16     673.919983
1996-12-17     663.349976
1996-12-18     661.390015
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5085, dtype: float64 Date
1996-12-12   -0.021712
1996-12-13   -0.000450
1996-12-16   -0.021986
1996-12-17   -0.015684
1996-12-18   -0.002955
                ...   
2017-12-21   -0.019649
2017-12-22    0.004408
2017-12-26   -0.005409
2017-12-27    0.003844
2017-12-28    0.012648
Name: Change1, Length: 5085, dtype: float64


In [6]:
kospi_test = fdr.DataReader('KS11', '2018-01-01')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1
#kospi_test['Change7'] = kospi_test['Close']/kospi_test['Close'].shift(7) - 1
#kospi_test['Change15'] = kospi_test['Close']/kospi_test['Close'].shift(15) - 1
#kospi_test['Change30'] = kospi_test['Close']/kospi_test['Close'].shift(30) - 1
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-02  2474.860107  2481.020020  2465.939941  2479.649902  2479.649902   
2018-01-03  2484.629883  2493.399902  2481.909912  2486.350098  2486.350098   
2018-01-04  2502.500000  2502.500000  2466.449951  2466.459961  2466.459961   
2018-01-05  2476.850098  2497.520020  2475.510010  2497.520020  2497.520020   
2018-01-08  2510.699951  2515.370117  2494.179932  2513.280029  2513.280029   

            Volume   Change1  
Date                          
2018-01-02  262200       NaN  
2018-01-03  331100  0.002702  
2018-01-04  333800 -0.008000  
2018-01-05  308800  0.012593  
2018-01-08  311400  0.006310  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')


In [7]:
kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-03  2484.629883  2493.399902  2481.909912  2486.350098  2486.350098   
2018-01-04  2502.500000  2502.500000  2466.449951  2466.459961  2466.459961   
2018-01-05  2476.850098  2497.520020  2475.510010  2497.520020  2497.520020   
2018-01-08  2510.699951  2515.370117  2494.179932  2513.280029  2513.280029   
2018-01-09  2510.729980  2524.729980  2498.409912  2510.229980  2510.229980   

            Volume   Change1  
Date                          
2018-01-03  331100  0.002702  
2018-01-04  333800 -0.008000  
2018-01-05  308800  0.012593  
2018-01-08  311400  0.006310  
2018-01-09  374300 -0.001214  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')


In [8]:
kospi_test_close = kospi_test['Close']
kospi_test_c1 = kospi_test['Change1']
#kospi_test_c7 = kospi_test['Change7']
#kospi_test_c15 = kospi_test['Change15']
#kospi_test_c30 = kospi_test['Change30']
print(kospi_test_close, kospi_test_c1)

Date
2018-01-03    2486.350098
2018-01-04    2466.459961
2018-01-05    2497.520020
2018-01-08    2513.280029
2018-01-09    2510.229980
                 ...     
2023-01-31    2425.080078
2023-02-01    2449.800049
2023-02-02    2468.879883
2023-02-03    2480.399902
2023-02-06    2447.100098
Name: Close, Length: 1253, dtype: float64 Date
2018-01-03    0.002702
2018-01-04   -0.008000
2018-01-05    0.012593
2018-01-08    0.006310
2018-01-09   -0.001214
                ...   
2023-01-31   -0.010361
2023-02-01    0.010193
2023-02-02    0.007788
2023-02-03    0.004666
2023-02-06   -0.013425
Name: Change1, Length: 1253, dtype: float64


In [9]:
kospi_train_close = kospi_train_close.values
kospi_train_c1 = 100 * kospi_train_c1.values
#kospi_train_c7 = 100 * kospi_train_c7.values
#kospi_train_c15 = 100 * kospi_train_c15.values
#kospi_train_c30 = 100 * kospi_train_c30.values

kospi_test_close = kospi_test_close.values
kospi_test_c1 = 100 * kospi_test_c1.values
#kospi_test_c7 = 100 * kospi_test_c7.values
#kospi_test_c15 = 100 * kospi_test_c15.values
#kospi_test_c30 = 100 * kospi_test_c30.values
print(kospi_test_c1)

[ 0.27020734 -0.7999733   1.25929711 ...  0.7788323   0.46660913
 -1.34251755]


In [10]:
lookback_window = 60
x_train_1, y_train_1 = [], []
#x_train_7, y_train_7 = [], []
#x_train_15, y_train_15 = [], []
#x_train_30, y_train_30 = [], []

In [11]:
for i in range(lookback_window, len(kospi_train_close)-1):
    x_train_1.append(kospi_train_close[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])

In [12]:
for index in range(len(y_train_1)):
    if y_train_1[index]>= -0.1 and y_train_1[index] < 0.1:
        y_train_1[index] = 'neutral'
    elif y_train_1[index] >= 0.1:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease' 

In [13]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
x_train_1 = x_train_1.reshape((5024, -1, 1))
#y_train_1 = y_train_1.reshape((-1, 1))

In [14]:
x_test_1, y_test_1 = [], []
#x_test_7, y_test_7 = [], []
#x_test_15, y_test_15 = [], []
#x_test_30, y_test_30 = [], []

In [15]:
for i in range(lookback_window, len(kospi_test_close)-1):
    x_test_1.append(kospi_test_close[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])

In [16]:
for index in range(len(y_test_1)):
    if y_test_1[index]>= -0.1 and y_test_1[index] < 0.1:
        y_test_1[index] = 'neutral'
    elif y_test_1[index] >= 0.1:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'

In [17]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
x_test_1 = x_test_1.reshape((1192, -1, 1))
#y_test_1 = y_test_1.reshape((-1, 1))

In [18]:
print(x_test_1.shape)
print(y_test_1.shape)

(1192, 60, 1)
(1192,)


In [19]:
print(y_test_1)

['neutral' 'neutral' 'decrease' ... 'increase' 'increase' 'increase']


In [20]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
#y_train_7_e = to_categorical(encoder.transform(y_train_7))
#y_train_15_e = to_categorical(encoder.transform(y_train_15))
#y_train_30_e = to_categorical(encoder.transform(y_train_30))

y_test_1_e = to_categorical(encoder.transform(y_test_1))
#y_test_7_e = to_categorical(encoder.transform(y_test_7))
#y_test_15_e = to_categorical(encoder.transform(y_test_15))
#y_test_30_e = to_categorical(encoder.transform(y_test_30))

In [21]:
print(y_test_1_e.shape)

(1192, 3)


In [22]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_4 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=5,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_4.summary()
model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_5 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=6,
        dilations=[1, 2, 4, 8, 16]
        ),
    Dense(3, activation='softmax')
])

model_5.summary()
model_5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-02-06 06:38:09.122562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 06:38:09.846325: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10698 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:d5:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 64)                74624     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 74,819
Trainable params: 74,819
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 64)                111552    
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 111,747
Trainabl

In [23]:
print('Train model_k2...')
model_1.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_1 = model_1.predict(x_test_1)

print('Train model_k3...')
model_2.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_2 = model_2.predict(x_test_1)

print('Train model_k4...')
model_3.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_3 = model_3.predict(x_test_1)

print('Train model_k5...')
model_4.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_4 = model_4.predict(x_test_1)

print('Train model_k6...')
model_5.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_5 = model_5.predict(x_test_1)

Train model_k2...
Epoch 1/100


2023-02-06 06:38:17.433113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2023-02-06 06:38:26.084456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-06 06:38:26.231074: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f503e25ca00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-06 06:38:26.231135: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): CUDA GPU, Compute Capability 8.6
2023-02-06 06:38:26.236161: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-06 06:38:26.378326: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 [==============================] - 69s 281ms/step - loss: 1193.7697 - accuracy: 0.4204
Epoch 2/100
157/157 [==============================] - 2s 15ms/step - loss: 394.0134 - accuracy: 0.4206
Epoch 3/100
157/157 [==============================] - 3s 16ms/step - loss: 356.2089 - accuracy: 0.4134
Epoch 4/100
157/157 [==============================] - 2s 15ms/step - loss: 179.5600 - accuracy: 0.4180
Epoch 5/100
157/157 [==============================] - 2s 15ms/step - loss: 186.6088 - accuracy: 0.4132
Epoch 6/100
157/157 [==============================] - 2s 15ms/step - loss: 133.9704 - accuracy: 0.4158
Epoch 7/100
157/157 [==============================] - 2s 15ms/step - loss: 110.1142 - accuracy: 0.4140
Epoch 8/100
157/157 [==============================] - 2s 15ms/step - loss: 115.8753 - accuracy: 0.4254
Epoch 9/100
157/157 [==============================] - 2s 15ms/step - loss: 97.2875 - accuracy: 0.4333
Epoch 10/100
157/157 [==============================] - 2s 15ms/step - los

157/157 [==============================] - 2s 15ms/step - loss: 3.7758 - accuracy: 0.4385
Epoch 81/100
157/157 [==============================] - 2s 15ms/step - loss: 3.7365 - accuracy: 0.4417
Epoch 82/100
157/157 [==============================] - 2s 14ms/step - loss: 3.9075 - accuracy: 0.4345
Epoch 83/100
157/157 [==============================] - 2s 15ms/step - loss: 3.2591 - accuracy: 0.4329
Epoch 84/100
157/157 [==============================] - 2s 15ms/step - loss: 3.3267 - accuracy: 0.4345
Epoch 85/100
157/157 [==============================] - 2s 15ms/step - loss: 3.0299 - accuracy: 0.4268
Epoch 86/100
157/157 [==============================] - 2s 15ms/step - loss: 2.5449 - accuracy: 0.4417
Epoch 87/100
157/157 [==============================] - 2s 15ms/step - loss: 3.3423 - accuracy: 0.4355
Epoch 88/100
157/157 [==============================] - 2s 16ms/step - loss: 2.2499 - accuracy: 0.4471
Epoch 89/100
157/157 [==============================] - 2s 15ms/step - loss: 2.8080 - 

157/157 [==============================] - 2s 15ms/step - loss: 6.0983 - accuracy: 0.4246
Epoch 59/100
157/157 [==============================] - 2s 15ms/step - loss: 4.9846 - accuracy: 0.4220
Epoch 60/100
157/157 [==============================] - 2s 16ms/step - loss: 4.5915 - accuracy: 0.4224
Epoch 61/100
157/157 [==============================] - 2s 15ms/step - loss: 4.6396 - accuracy: 0.4264
Epoch 62/100
157/157 [==============================] - 2s 15ms/step - loss: 5.1944 - accuracy: 0.4232
Epoch 63/100
157/157 [==============================] - 2s 15ms/step - loss: 4.0325 - accuracy: 0.4283
Epoch 64/100
157/157 [==============================] - 2s 15ms/step - loss: 3.8851 - accuracy: 0.4192
Epoch 65/100
157/157 [==============================] - 2s 15ms/step - loss: 6.3156 - accuracy: 0.4258
Epoch 66/100
157/157 [==============================] - 2s 16ms/step - loss: 5.5924 - accuracy: 0.4232
Epoch 67/100
157/157 [==============================] - 2s 15ms/step - loss: 3.4583 - 

157/157 [==============================] - 2s 15ms/step - loss: 7.8927 - accuracy: 0.4262
Epoch 37/100
157/157 [==============================] - 2s 15ms/step - loss: 8.0579 - accuracy: 0.4216
Epoch 38/100
157/157 [==============================] - 2s 15ms/step - loss: 5.4996 - accuracy: 0.4291
Epoch 39/100
157/157 [==============================] - 3s 16ms/step - loss: 7.8189 - accuracy: 0.4246
Epoch 40/100
157/157 [==============================] - 2s 15ms/step - loss: 7.1127 - accuracy: 0.4359
Epoch 41/100
157/157 [==============================] - 2s 15ms/step - loss: 4.4461 - accuracy: 0.4425
Epoch 42/100
157/157 [==============================] - 2s 16ms/step - loss: 4.6704 - accuracy: 0.4214
Epoch 43/100
157/157 [==============================] - 3s 17ms/step - loss: 4.5025 - accuracy: 0.4270
Epoch 44/100
157/157 [==============================] - 2s 16ms/step - loss: 4.5437 - accuracy: 0.4385
Epoch 45/100
157/157 [==============================] - 2s 16ms/step - loss: 3.9892 - 

157/157 [==============================] - 2s 15ms/step - loss: 28.7394 - accuracy: 0.4124
Epoch 15/100
157/157 [==============================] - 2s 15ms/step - loss: 24.5256 - accuracy: 0.4240
Epoch 16/100
157/157 [==============================] - 2s 15ms/step - loss: 26.1644 - accuracy: 0.4178
Epoch 17/100
157/157 [==============================] - 2s 16ms/step - loss: 22.6993 - accuracy: 0.4172
Epoch 18/100
157/157 [==============================] - 2s 15ms/step - loss: 23.1201 - accuracy: 0.4275
Epoch 19/100
157/157 [==============================] - 2s 15ms/step - loss: 17.8286 - accuracy: 0.4248
Epoch 20/100
157/157 [==============================] - 2s 16ms/step - loss: 60.4044 - accuracy: 0.4158
Epoch 21/100
157/157 [==============================] - 2s 14ms/step - loss: 17.2292 - accuracy: 0.4112
Epoch 22/100
157/157 [==============================] - 3s 16ms/step - loss: 18.0226 - accuracy: 0.4152
Epoch 23/100
157/157 [==============================] - 2s 15ms/step - loss: 

157/157 [==============================] - 2s 15ms/step - loss: 0.9333 - accuracy: 0.4827
Epoch 94/100
157/157 [==============================] - 2s 16ms/step - loss: 0.9332 - accuracy: 0.4827
Epoch 95/100
157/157 [==============================] - 3s 16ms/step - loss: 0.9331 - accuracy: 0.4827
Epoch 96/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9331 - accuracy: 0.4827
Epoch 97/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9331 - accuracy: 0.4827
Epoch 98/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9333 - accuracy: 0.4827
Epoch 99/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9332 - accuracy: 0.4827
Epoch 100/100
38/38 [==============================] - 4s 90ms/step
Train model_k6...
Epoch 1/100
157/157 [==============================] - 15s 15ms/step - loss: 928.2712 - accuracy: 0.4198
Epoch 2/100
157/157 [==============================] - 3s 16ms/step - loss: 196.7932 - accuracy: 0.41

157/157 [==============================] - 2s 16ms/step - loss: 0.9336 - accuracy: 0.4783
Epoch 72/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9334 - accuracy: 0.4827
Epoch 73/100
157/157 [==============================] - 3s 16ms/step - loss: 0.9331 - accuracy: 0.4827
Epoch 74/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9336 - accuracy: 0.4827
Epoch 75/100
157/157 [==============================] - 2s 16ms/step - loss: 0.9333 - accuracy: 0.4827
Epoch 76/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9332 - accuracy: 0.4827
Epoch 77/100
157/157 [==============================] - 2s 16ms/step - loss: 0.9334 - accuracy: 0.4827
Epoch 78/100
157/157 [==============================] - 3s 16ms/step - loss: 0.9334 - accuracy: 0.4827
Epoch 79/100
157/157 [==============================] - 3s 16ms/step - loss: 0.9334 - accuracy: 0.4827
Epoch 80/100
157/157 [==============================] - 2s 15ms/step - loss: 0.9332 - 

In [ ]:
model_1.save('/home/work/nlp/TCN_models/classification_model/1day/kospi_1day_classification_k2d16.h5')
model_2.save('/home/work/nlp/TCN_models/classification_model/1day/kospi_1day_classification_k3d16.h5')
model_3.save('/home/work/nlp/TCN_models/classification_model/1day/kospi_1day_classification_k4d16.h5')
model_4.save('/home/work/nlp/TCN_models/classification_model/1day/kospi_1day_classification_k5d16.h5')
model_5.save('/home/work/nlp/TCN_models/classification_model/1day/kospi_1day_classification_k6d16.h5')

In [ ]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_2.evaluate(x_test_1, y_test_1_e)[1]
model_3.evaluate(x_test_1, y_test_1_e)[1]
model_4.evaluate(x_test_1, y_test_1_e)[1]
model_5.evaluate(x_test_1, y_test_1_e)[1]